In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import dill as pickle

In [ ]:
def load_job_result(file_name):
    with open(file_name, mode="rb") as f:
        jr = pickle.load(f)
    
    jr.N_models = len(jr.description["objective_func"])
    
    return jr

In [ ]:
jr = load_job_result("output_0_0.pickle")

In [ ]:
def plot_minimum_objective(jr):
    _, dataY = jr.test_repository.get(jr.test_suite)
    dataY = np.asarray(dataY)
    component_minima = [[] for i in range(dataY.shape[1])]
    total_minima = []
    for output in dataY:
        for i in range(len(output)):
            if len(component_minima[i]) == 0 or output[i] < component_minima[i][-1]:
                component_minima[i].append(output[i])
            else:
                component_minima[i].append(component_minima[i][-1])
                
        min_obj = np.min(output)
        if len(total_minima) == 0 or min_obj < total_minima[-1]:
            total_minima.append(min_obj)
        else:
            total_minima.append(total_minima[-1])
    
    fig, axs = plt.subplots(1, len(output) + 1, sharex=True, sharey=True, figsize=(10*len(output), 10))
    for i in range(len(output)):
        axs[i].plot(component_minima[i])
        axs[i].set_title("Component {} minimum".format(i+1))
    axs[-1].plot(total_minima)
    axs[-1].set_title("Overall minimum")

In [ ]:
def plot_model_losses(jr, model):
    perf = jr.model_performance[model]
    data_D = perf.get_history("discriminator_loss")
    data_G = perf.get_history("generator_loss")
    rows = len(data_D)
    
    fig, axs = plt.subplots(rows, 2, sharex=True, figsize=(10, 10*rows))
    for r in range(rows):
        idx = (r,0) if rows > 1 else 0
        axs[idx].plot(data_D[r])
        axs[idx].set_title("Model reset {} discriminator l.".format(r))
        idx = (r,1) if rows > 1 else 1
        axs[idx].plot(data_G[r])
        axs[idx].set_title("Model reset {} generator l.".format(r))
        
    

In [ ]:
plot_model_losses(jr, 0)

In [ ]:
plot_minimum_objective(jr)

In [ ]:
def first_falsified_at(jr):
    _, outputs = jr.test_repository.get(jr.test_suite)
    outputs = np.asarray(outputs)
    
    falsified = False
    for i in range(outputs.shape[0]):
        if np.min(outputs[i,:]) == 0:
            falsified = True
            break
    
    return i + 1 if falsified else -1

In [ ]:
def first_falsifications(jrs):
    return [first_falsified_at(jr) for jr in jrs]

def falsification_rate(jrs):
    return sum(1 for jr in jrs if first_falsified_at(jr) > 0)/len(jrs)

In [ ]:
models = []
for i in range(2):
    jrs = []
    for j in range(0, 10):
        jrs.append(load_job_result("job_{}_{}.pickle".format(i, j)))
    models.append(jrs)

In [ ]:
for i in range(len(models)):
    print("Model {}:".format(i))
    print("  falsifications: {}".format(first_falsifications(models[i])))
    print("  falsification rate: {}".format(falsification_rate(models[i])))